In [2]:
# Import the OpenAI library. This library provides a Python interface to the OpenAI API.
from openai import OpenAI
import sounddevice as sd
import numpy as np
from vosk import Model, KaldiRecognizer
import json

In [8]:
# Create a Recognizer instance
r = sr.Recognizer()

# Create an instance of the OpenAI client. The base_url parameter is set to a local server and the api_key is not needed in this case.
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

# Define the conversation history
history = [
    {"role": "system", "content": "I'm going to give you an idea, a prompt for an AI image generator and you are going to enhance that prompt by adding descriptions and keywords"},
]

# Use the microphone as the audio source using a with statement because otherwise sr.Microphone() throws an error
while True:
    with sr.Microphone() as source:
        print("Say 'Listen AI' to start listening!")
        audio = r.listen(source)

        try:
            # Use Google's Speech Recognition API to convert the audio to text
            text = r.recognize_google(audio)
            print("You said: " + text)

            if text.lower() == "listen ai":
                print("Listening for your command!")
                audio = r.listen(source)
                text = r.recognize_google(audio)
                history.append({"role": "user", "content": text})
                print("Your command: " + text)


                # Call the chat.completions.create method of the client. This method is used to generate a chat completion.
                # The model parameter is set to "local-model", but it's currently unused.
                # The messages parameter is the conversation history. 
                # The temperature parameter is set to 0.7. This parameter controls the randomness of the model's output. A higher value makes the output more random, while a lower value makes it more deterministic.
                completion = client.chat.completions.create(
                    model="local-model", 
                    messages=history,
                    temperature=0.7,
                )

                # Print the message of the first choice from the completion. The choices attribute of a completion contains the different outputs generated by the model.
                print(completion.choices[0].message)
                history.append({"role": "assistant", "content": completion.choices[0].message['content']})


        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))


Say something!
You said: hello how is your day hello
Say something!


KeyboardInterrupt: 

In [1]:

# Set the sample rate and the number of channels
sample_rate = 16000
channels = 1

# Load the model
model = Model("models/vosk-model-en-us-0.22")

# Create a recognizer
rec = KaldiRecognizer(model, sample_rate)

# Define a callback function to process the audio chunks
def callback(indata, frames, time, status):
    if rec.AcceptWaveform(indata.flatten().tobytes()):
        result = rec.Result()
        result = json.loads(result)
        if result.get('text'):  # Only print when there is text
            print("Transcription: ", result['text'])

# Create a stream to record audio
stream = sd.InputStream(callback=callback, channels=channels, samplerate=sample_rate, dtype='int16')

# Start recording and processing
with stream:
    print("Recording and transcribing. Press Ctrl+C to stop.")
    while True:
        sd.sleep(200)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:11:12:13:14:15
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from models/vosk-model-en-us-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from models/vosk-model-en-us-0.22/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:297) Loading words from models/vosk-model-en-us-0.22/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo models/vosk-model-en-us-0.22/graph/phones/word_boundary.int
LOG (VoskAPI:ReadDataFiles():mod

Recording and transcribing. Press Ctrl+C to stop.
Transcription:  well hello there this is pretty amazing i could not instruct my model to generate the stuff i want to and then it will have context and then i can make it into history and then after that i can just tell him what to change and it will change it
Transcription:  and then i have a very nice speech to text with combined other lamps and i image to write that that will be great


KeyboardInterrupt: 